# Testy statystyczne. Część II

In [12]:
import pandas as pd
from pandas import DataFrame, Series
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg
from pprint import pprint

## Test znaków rangowych Wilcoxona / Test U Manna-Whitneya

### Przykład

Grupa badaczy postanowiła przeprowadzić badanie nawiązujące do słynnego badania Kahnemana i Tverskyego. W oryginalnym badaniu naukowcy przedstawiali badanym krótką historykę:
> Linda ma 31 lat, jest otwartą, inteligentną, i niezamężną kobietą. Ukończyła filozofię. Jako studentka poświęcała dużo czasu problemom sprawiedliwości społecznej i dyskryminacji, uczestniczyła też w demonstracjach antynuklearnych. Co jest bardziej prawdopodobne?

po której prosili o wskazanie bardziej prawdopodobnego z dwóch stwierdzeń:

1. Linda pracuje w banku
2. Linda pracuje w banku i jest aktywną działaczką ruchu feministycznego

Badani znacznie częściej wybierali zdanie numer 2, mimo, że *a priori* prawdopodobieństwo iloczynu dwóch zdarzeń nie może być wyższe niż prawdopodobieństwo wystąpienia któregokolwiek z nich pojedynczo. 

W naszym przykładzie badacze chcieli sprawdzić, jak ludzie oceniają prawdopodobieństwo każdego z tych zdań bez kontrastu. W tym celu przedstawiali dwóm grupom (N=40 dla każdej z nich) badanych tę samą historyjkę i pokazywali jedno pytanie. Zadaniem badanych było ocenienie jak bardzo prawdopodobne są te stwierdzenia na skali siedmiostopniowej Likerta (1=zupełnie nieprawdopodobne, 7=prawie pewne). Wyniki eksperymentu znajdują się w pliku `linda.csv`. Czy mediana obu rozkładów różni się od siebie? Żeby to sprawdzić użyjemy testu U Manna-Whitneya.

In [13]:
data = pd.read_csv('linda.csv')
data.head()

,Unnamed: 0,participant,group,likelihood
0,0,1,bank_teller,3
1,0,2,bank_teller,1
2,0,3,bank_teller,1
3,0,4,bank_teller,2
4,0,5,bank_teller,1


Standardowo podzielimy nasza ramkę na dwie części i wydobędziemy właściwą kolumnę.

In [14]:
bank_teller_resp = data[data['group'] == 'bank_teller']['likelihood']
bank_teller_resp.median()

3.0

In [15]:
bank_teller_and_fem_resp = data[data['group'] == 'bank_teller_and_fem']['likelihood']
bank_teller_and_fem_resp.median()

5.0

W module `scipy.stats` znajduje się funkcja `mannwhitneyu`, za pomocą której możemy przeprowadzić właściwy test. Jako argumenty przyjmuje `Series` (lub inne listopodobne obiekty) i zwraca wartośc statystyki testowej U oraz p-wartość.

In [16]:
u, p = stats.mannwhitneyu(bank_teller_resp, bank_teller_and_fem_resp)
print('Statystyka testowa U: ', u)
print('Wartość p: ', p)

Statystyka testowa U:  213.0
Wartość p:  5.023207721468035e-09


Odpowiednik tej funkcji możemy także znaleźć w pakiecie `pingouin`.

In [17]:
pg.mwu(bank_teller_resp, bank_teller_and_fem_resp)

,U-val,p-val,RBC,CLES
MWU,213.0,1.004642e-08,0.73375,0.811875


Na podstawie przeprowadzonego testu mamy więc prawo odrzucić hipotezę zerową (przy ustalonym poziomie istotności statystycznej $\alpha = 0.05$).

## Test Kruskala-Wallisa

### Przykład

Grupa badaczy postanowiła zbadać interpretację rzeczowników w takich zdaniach jak:

> Chłopcy palą papierosy

Możliwe są dwie interpretacje:
- MAKSYMALNA: Wszyscy chłopcy (w danej domenie odniesienia) palą papierosy
- MINIMALNA: Istnieje przynajmniej dwóch chłopców (w danej domenie odniesienia), którzy palą papierosy

Aby przetestować tę hipotezę przeprowadzili eksperyment. Uczestnikom badania prezentowali krótką historyjkę:

> Marek, Jacek i Piotrek po zakończeniu lekcji poszli do małego zagajnika za szkołą. Marek wyciągnął z kieszeni paczkę papierosów i poczęstował kolegów. Jacek wziął papierosa, ale Piotrek odmówił, mówiąc, że papierosy szkodzą. Potem Marek i Jacek wypalili swoje papierosy i wszyscy trzej chłopcy rozeszli się do domów.

Następnie badacze prosili uczestników o wyrażenie na 10-stopniowej skali Likerta swojej zgody na jedno z trzech zdań (w zależności od warunku eksperymentalnego):

- (no-quantifier) Chłopcy palą papierosy
- (universal quantifier) Wszyscy chłopcy palą papierosy
- (existential quantifier) Jacyś chłopcy palą papierosy

Ich hipoteza głosiła, że zdania bez kwantyfikatora będą interpretowane tak jak zdania z kwantyfikatorem uniwersalnym, ich interpretacja będzie zaś różnić się od zdań z kwantyfikatorem egzystencjalnym. Dane z eksperymentu znajdują się w pliku `elektryczne_gitary.csv`. Czy dane wspierają hipotezę badaczy? Użyjmy testu Kruskala-Wallisa (znanego także jako  "nieparametryczna ANOVA").

In [18]:
data = pd.read_csv('elektrycznie_gitary.csv')
data.head(6)

,participant,condition,sent_judg
0,1,universal,3
1,2,existential,7
2,3,no-quantifier,5
3,4,universal,3
4,5,existential,6
5,6,no-quantifier,4


Po wczytaniu danych musimy podzielić nasze dane na 3 warunki eksperymentalne oraz wydobyć z nich kolumnę, w której znajdują się oceny zdań.

In [19]:
universal = data[data['condition'] == 'universal']['sent_judg']
existential = data[data['condition'] == 'existential']['sent_judg']
no_quantifier = data[data['condition'] == 'no-quantifier']['sent_judg']

Zobaczmy jak wyglądają statystyki deskryptywne dla naszych trzech warunków:

In [20]:
data.groupby('condition')['sent_judg'].describe()

,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
existential,40.0,6.075,1.118321,3.0,6.00,6.0,7.00,7.0
no-quantifier,40.0,4.475,1.240089,2.0,3.75,4.5,6.00,6.0
universal,40.0,1.950,1.060962,1.0,1.00,2.0,2.25,5.0


Widzimy, że oceny zdań bez kwantyfikatora znajdują się mniej-więcej pomiędzy tymi z kwantyfikatorem egzystencjalnym oraz kwantyfikatorem uniwersalnym. Przetestujmy, czy różnice te są statystycznie istotne za pomocą testu Kruskala-Wallisa (nieparametryczny odpowiednik analizy wariancji). Możemy to zrobić za pomocą funkcji `kruskal` z modułu `scipy.stats`. Jako argumenty przyjmuje ona $n$ list (lub obiektów listo-podobnych) i zwraca dwie wartości - statystykę testową oraz p-wartość.

In [21]:
h, p = stats.kruskal(universal, existential, no_quantifier)
print('Statystyka testowa H: ', h)
print('Wartość p: ', p)

Statystyka testowa H:  81.86698909549435
Wartość p:  1.6703565825911476e-18


Test Kruskala-Wallisa dał statystycznie istotny wynik. To samo możemy osiągnąć posługując się funkcją `kruskal` z pakietu `pingouin`. Jedyna różnica to fakt, że wyniki są przyjemnie dla oka sformatowane. Jako `dv` przekazujemy nazwę kolumny ze zmienną zależną, jako `between` nazwę kolumny ze zmienną niezależną.

In [22]:
pg.kruskal(dv = 'sent_judg', between = 'condition', detailed = True, data = data)

,Source,ddof1,H,p-unc
Kruskal,condition,2,81.867,1.670357e-18


Potrzebujemy jeszcze wykonać test *post-hoc*, który powie nam, które warunki eksperymentalne różnią się od których w statystycznie istotny sposób. Jako test *post-hoc* wykonamy porównanie parami wyników w naszych warunkach eksperymentalnych. Najładniejsze *output* generuje funkcja `pairwise_ttests` z pakietu `pingouin`. Aby skorzystać z tej funkcji musimy przekazać nam ramkę danych z danymi (argument `data`), nazwę kolumny ze zmienną zależną (argument `dv`) oraz nazwę kolumny ze zmienną grupującą (argument `between`). Dodatkowo możemy od razu zastosować poprawkę na wielokrotne testowanie - ja wybrałem standardową poprawkę Bonferroniego. 

In [23]:
pg.pairwise_ttests(dv = 'sent_judg', between='condition', data = data, padjust = 'bonferroni')

,Contrast,A,B,Paired,T,tail,p-unc,p-corr,p-adjust,BF10,efsize,eftype
0,condition,universal,existential,False,-16.924,two-sided,7.840350e-28,2.352105e-27,bonferroni,2.427394e+24,-3.747833,hedges
1,condition,universal,no-quantifier,False,-9.785,two-sided,3.267767e-15,9.803300e-15,bonferroni,1.355837e+12,-2.166923,hedges
2,condition,existential,no-quantifier,False,6.060,two-sided,4.532599e-08,1.359780e-07,bonferroni,2.363720e+05,1.341971,hedges


Okazuje się, że we wszystkich trzech porównaniach otrzymaliśmy statystycznie istotny wynik!

## Korelacja między zmiennymi

Wróćmy do badania dotyczącego wpływu ryzyka pomyłki na akceptacje zdań przypisujących wiedzę. Jedną z hipotez wysuwanych w literaturze jest hipoteza stwierdzająca, że wpływ ten jest związany z wiekiem. Osoby starsze z natury są bardziej ostrożne, co skutkuje wstrzemięźliwością w przypisywaniu wiedzy. Badacze postanowili więc przedstawić badanym ($N=100$) krótką historyjkę: 

> Maria spytała Jacka: "Jacku, czy wiesz która jest godzina?". Jacek spojrzał na zegarek i odrzekł "Mario, jest godzina 14:20".

> Jacek bardzo rzadko korzysta z zegarka i nie sprawdzał bardzo dawno czy się nie spieszy czy się nie spóźnia. Dodatkowo należy zauważyć, że patrzył na zegarek tylko pół sekundy i nie sprawdził wcześniej, czy nie ma go założonego odwrotnie.

Następnie poprosili badanych o wyrażenie zgody (na 10-stopniowej skali Likerta) na zdanie:

> Jacek wie, że jest godzina 14:20

Badacze oczywiście zbierali podstawowe dane demograficzne, w tym wiek. Wyniki eksperymentu znajdują się w pliku `risk.csv`. Czy dane wspierają hipotezę o związku wieku i sposobu przypisywania wiedzy?

### Przykład

In [25]:
data = pd.read_csv('risk.csv')

Zobaczmy jak wyglądają nasze dane. Nasza uwagę powinno zwrócić to, że mieliśmy uczestników nawet 11 letnich (!), a najstarszy uczestnik miał tylko 43 lata. Kwartyle sugerują, że, jeśli chodzi o wiek, mamy dość mały rozrzut obserwacji. Prawdopodobnie wyniki zbierane były głównie wśród studentów. Trudno. 

In [26]:
data[['age', 'rating']].describe()

,age,rating
count,100.000000,100.000000
mean,26.330000,5.130000
std,7.473935,1.883769
min,11.000000,1.000000
25%,21.000000,4.000000
50%,27.000000,5.000000
75%,31.000000,6.000000
max,43.000000,10.000000


Chcielibyśmy obliczyć współczynnik korelacji między dwiema zmiennymi - wiekiem oraz zgodą na zdanie dotyczące wiedzy - oraz sprawdzić, czy korelacja ta jest statystycznie istotna. W tym celu możemy posłużyć się funkcją `pearsonr` z modułu `scipy.stats` (inne standardowe współczynniki korelacji są również dostępne). Przyjmuje ona dwie listy z wartościami, między którymi chcemy obliczyć korelację.

In [27]:
r, p = stats.pearsonr(data['age'], data['rating'])
print('Współczynnik korelacji r Pearsona: ', r)
print('Wartość p: ', p)

Współczynnik korelacji r Pearsona:  0.35923126201089883
Wartość p:  0.00024211240190663978


Widzimy, że korelacja jest statystycznie istotna. Odpowiednikiem funkcji `scipy.stats.pearsonr` w pakiecie `pingouin` jest funkcja `corr`. Pozwala ona (argument `method` obliczyć różne inne współczynniki korelacji. W tym wypadku obliczymy Tau Kendalla oraz r Spearmana.

In [28]:
pg.corr(data['age'], data['rating'], method = 'kendall')

,n,r,CI95%,r2,adj_r2,p-val,power
kendall,100,0.287,"[0.1, 0.46]",0.082,0.063,0.000099,0.831


In [29]:
pg.corr(data['age'], data['rating'], method = 'pearson')

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,100,0.359,"[0.18, 0.52]",0.129,0.111,0.000242,64.716,0.961


In [30]:
pg.corr(data['age'], data['rating'], method = 'spearman')

,n,r,CI95%,r2,adj_r2,p-val,power
spearman,100,0.383,"[0.2, 0.54]",0.147,0.129,0.000083,0.979
